<a href="https://colab.research.google.com/github/feiduobaby/good-first-issue/blob/main/Homework9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
PREFIX="https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL=f"{PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL=f"{PREFIX}/hair_classifier_v1.onnx"
!wget $DATA_URL
!wget $MODEL_URL

--2025-12-02 12:32:43--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/398ded4a-c41c-4e5a-9672-acb7e441de54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-02T13%3A15%3A03Z&rscd=attachment%3B+filename%3Dhair_classifier_v1.onnx.data&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-02T12%3A14%3A50Z&ske=2025-12-02T13%3A15%3A03Z&sks=b&skv=2018-11-09&sig=6vAlRhQX2uIAQxw89JGRfdS%2FiX6E2wmF9JDwAiFI5Co%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDY4MDU2MywibmJmIjoxNzY0Njc4NzYzLCJwY